# ДЗ по SELENIUM

## Первая часть кода открывает сайт IMDB и выгружает список топ250 фильмов

Это выполняет домашннее задание по SQL:

Создайте sql-таблицу об известных людях, странах и т.д. В ней должно содержаться не менее 5 колонок с типами данных (text, float, integer) и не менее трех строк. Сохраните таблицу в файле с расширением .sqlite или .sqlite3. 

## Вторая часть - используя Selenium переходит по каждой из ссылок и сохраняет информацию о каждом фильме из списка, после этого находит того, кто снял больше всего фильмов


In [1]:
import os
import re
import sqlite3

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver as wb
from time import sleep

In [2]:

driver = wb.Chrome('../chromedriver_win32/chromedriver.exe')

driver.implicitly_wait(2)
driver.get("https://www.imdb.com/chart/top/?ref_=nv_mv_250")

soup = BeautifulSoup(driver.page_source, 'lxml')
df_250 = pd.DataFrame(columns=['title', 'year', 'rating', 'rating_count', 'link'])

table_films = soup.find_all('tbody', {'class': 'lister-list'})[0]
for elem in table_films.find_all('tr'):
    title_cell = elem.find_all('td', {'class': 'titleColumn'})[0]
    title = title_cell.a.text
    #     print(title)
    link = title_cell.find('a').get('href')
    #     print(link)
    date_text = elem.find_all('td', {'class': 'titleColumn'})[0].span.text
    date_int = int(re.findall(r'\d+', date_text)[0])
    #     print(date_int)
    rating_cell = elem.find_all('td', {'class': 'ratingColumn imdbRating'})[0]
    rating = float(rating_cell.text.strip().replace(',', '.'))
    count_info = (rating_cell.find('strong').get('title'))
    num_rating = (re.findall(r'(?:\d+\s)+(?:\d+)', count_info))
    num_rating = int("".join(num_rating[0].split()))
    #     print(num_rating)
    #     print(rating)
    row_to_add = [title, date_int, rating, num_rating, link]
    #     print(row_to_add)
    our_one_row = pd.DataFrame([row_to_add], columns=['title', 'year', 'rating', 'rating_count', 'link'])
    df_250 = df_250.append(our_one_row, ignore_index=True)  # add it to a temporary df

if not os.path.exists('movie.sqlite'):
    conn = sqlite3.connect('movie.sqlite')
    cur = conn.cursor()
    # crating a new table with our structure
    cur.executescript(
        '''CREATE TABLE "top250" (
        "title" TEXT,
        "year"  INTEGER,
        "rating"   NUMERIC,
        "rating_count"  INTEGER,
        "link"    TEXT,
        PRIMARY KEY (title, year));
    ''')
    conn.commit()
    cur.close()

# open connection and write from pandas to sqlite
conn = sqlite3.connect('movie.sqlite')
df_250.to_sql('top250', conn, if_exists='replace', index=False)
conn.commit()


## Проверим, записалось ли что-то в БД

In [3]:
pd.read_sql('select * from top250 limit 5', conn)

,title,year,rating,rating_count,link
0,Побег из Шоушенка,1994,9.2,2298438,/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...
1,Крестный отец,1972,9.1,1586270,/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...
2,Крестный отец 2,1974,9.0,1108300,/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...
3,Темный рыцарь,2008,9.0,2262484,/title/tt0468569/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...
4,12 разгневанных мужчин,1957,8.9,675343,/title/tt0050083/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...


## Задача моя на задание - найти какие режиссеры сняли больше всего фильмов из топ250

In [4]:
def get_director_by_n_film(topn):
    chr_driver = wb.Chrome('../chromedriver_win32/chromedriver.exe')
    chr_driver.implicitly_wait(1)
    chr_driver.get("https://www.imdb.com/chart/top/?ref_=nv_mv_250")

    film_link = f'//*[@id="main"]/div/span/div/div/div[3]/table/tbody/tr[{topn}]/td[2]/a'
    fld = chr_driver.find_element_by_xpath(film_link)
    fld.click()
    try:
        director = chr_driver.find_element_by_xpath('//*[@id="title-overview-widget"]/div[2]/div[1]/div[2]/a').text
    except:
        director = chr_driver.find_element_by_xpath(
            '//*[@id="title-overview-widget"]/div[2]/div[2]/div[1]/div[2]/a').text

    chr_driver.quit()
    return director


conn = sqlite3.connect('movie.sqlite')  # friends db
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Directors (id INTEGER PRIMARY KEY, 
                                                    director TEXT)''')


Проверим на одном

In [5]:
get_director_by_n_film(134)

'Akira Kurosawa'

In [6]:
positions = range(3)
# positions = range(250) ## - нужно запустить эту строку для обновления всех данных, работает медленно

for pos in positions:
    try:
        cur_director = get_director_by_n_film(pos + 1)
    except:
        cur_director = ""
    cur.execute('''REPLACE INTO Directors
                            (id, director) VALUES (?, ?)''', (pos + 1, cur_director))
    conn.commit()
    print(pos + 1, cur_director)


1 Frank Darabont
2 Francis Ford Coppola
3 Francis Ford Coppola


In [7]:
top_directors = cur.execute(
    "SELECT director, count(*) from directors GROUP by director ORDER by count(*) DESC LIMIT 10")
top10 = pd.DataFrame(top_directors, columns=['Director', 'number of films'])
top10

,Director,number of films
0,Stanley Kubrick,7
1,Martin Scorsese,7
2,Christopher Nolan,7
3,Steven Spielberg,6
4,Charles Chaplin,6
5,Alfred Hitchcock,6
6,Akira Kurosawa,6
7,Quentin Tarantino,5
8,Hayao Miyazaki,5
9,Billy Wilder,5


Первое место делят:

In [8]:
top10[top10['number of films'] == max(top10['number of films'])]

,Director,number of films
0,Stanley Kubrick,7
1,Martin Scorsese,7
2,Christopher Nolan,7


In [9]:
conn.close()